In [55]:
import os
import time 
import pickle

import numpy as np
from copy import deepcopy

import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "plotly_white"

In [56]:
# Root of Project
ROOT = '/home/dev/ws_percept/src'

# Cameras
IMAGE_SIZE  = 240
CAMERAS = ['cam1', 'cam2', 'cam3']

# Scene
# [x_min, y_min, z_min, x_max, y_max, z_max] - the metric volume to be voxelized
SCENE_BOUNDS    = (-1.5, -1.5, -1.5, 1.50, 1.5, 1.5)

# Agents
AGENTS = ['panda0', 'panda1']

SHOW_CPH_VIZ = False

In [57]:
# load data
DATASET_PATH = '../outputs/dualarms_3cam/dualarms_3cam_with_fk.pkl'
with open(DATASET_PATH, 'rb') as f:
    _obs = pickle.load(f)

## Setup PCD, Mesh

In [58]:
def constrain_scene_bounds(data):
    global SCENE_BOUNDS

    nX, nY, nZ, X, Y, Z = SCENE_BOUNDS

    X_mask = (data[:,0] > nX) & (data[:,0]<X)
    Y_mask = (data[:,1] > nY) & (data[:,1]<Y)
    Z_mask = (data[:,2] > nZ) & (data[:,2]<Z)
    mask = (X_mask)&(Y_mask)&(Z_mask) 

    return data[mask]

def get_pcd_and_rgb(camera_obs:dict):
    pcd = camera_obs['pointcloud'].reshape(-2,3)
    rgb = camera_obs['rgb'].reshape(-2,3)
    return pcd, rgb

def get_pcd(camera_obs:dict):
    pcd = camera_obs['pointcloud'].reshape(-2,3)
    return pcd


obs = _obs[-1]
camera_obs = obs[CAMERAS[0]]
pcd, rgb = get_pcd_and_rgb(camera_obs)

In [59]:
from scipy.spatial.transform import Rotation as R
import cupoch as cph

ROBOT_URDF_PATH = 'outputs/testdata/franka_panda_cupoch/panda.urdf'
URDF_JOINT_PREFIX = 'panda_joint'

path = os.path.join(ROOT, ROBOT_URDF_PATH)
kin = cph.kinematics.KinematicChain(path)


def get_tf_matrix(
    translation:np.array,
    rotation:np.array, 
) -> np.array:

    r = R.from_rotvec(rotation)
    r = r.as_matrix()

    tf = np.eye(4)
    tf[:3,:3] = r
    tf[:3,3] = translation

    return tf

def manually_offset_base(ref:np.array, translation_offset) -> np.array:
    offset = np.eye(4)
    offset[:3, 3] = translation_offset
    return np.dot(ref, offset)

def get_mesh_using_forward_kinematics(
    joints_positions:np.array,
    global_position:np.array,
    global_rotation:np.array
):
    global URDF_JOINT_PREFIX

    # create joint_map, used for fk on urdf model
    joint_map = {'%s%d' % (URDF_JOINT_PREFIX, i+1): val for i, val \
                in enumerate(joints_positions)} 

    # create transformation matrix
    tf_matrix = get_tf_matrix(global_position, global_rotation)

    # manual offset, to be removed 
    offset = np.array([0.04, 0.0, -0.068])
    tf_matrix = manually_offset_base(tf_matrix, offset)
    
    # do forward kinematics
    poses = kin.forward_kinematics(joint_map, tf_matrix)
    
    # store mesh geometries
    meshes = kin.get_transformed_visual_geometry_map(poses)
    
    return list(meshes.values())

## PCR

In [60]:
def get_pcds_for_registration(obs):
    global CAMERAS

    pcds = list()
    for CAMERA in CAMERAS:
        camera_obs = obs[CAMERA]
        pcd, rgb = get_pcd_and_rgb(camera_obs)
        pcd = constrain_scene_bounds(pcd)
        pcds.append(pcd)

    return pcds

pcds = get_pcds_for_registration(obs)

In [61]:
def do_point_cloud_registration(pcds):
    assert len(pcds) > 1
    def register(
        source_gpu:cph.geometry.PointCloud, 
        target_gpu:cph.geometry.PointCloud,
    ) -> cph.geometry.PointCloud:

        threshold = 0.02 # what does this do?
        trans_init = np.asarray([[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0],
                                [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0]])

        # register pointclouds
        start = time.time()
        reg_p2p = cph.registration.registration_icp(
            source_gpu,
            target_gpu,
            threshold,
            trans_init.astype(np.float32),
            cph.registration.TransformationEstimationPointToPlane(),
        )

        source_gpu.transform(reg_p2p.transformation)

        # remove outliers
        NEIGHBOURS = 2
        source_gpu, ind = source_gpu.remove_statistical_outlier(nb_neighbors=NEIGHBOURS, std_ratio=2.0)
        target_gpu, ind = target_gpu.remove_statistical_outlier(nb_neighbors=NEIGHBOURS, std_ratio=2.0)


        elapsed_time = time.time() - start
        print("ICP (GPU) [sec]:", elapsed_time) # adding outlier removal adds ~25ms
    
        return source_gpu+target_gpu


    # load first and second pcd
    source_gpu = cph.geometry.PointCloud(
        cph.utility.HostVector3fVector(pcds[0])
    )
    target_gpu = cph.geometry.PointCloud(
        cph.utility.HostVector3fVector(pcds[1])
    )
    source_gpu = register(source_gpu, target_gpu)

    # merge all other pcds with new source
    for pcd in pcds[2:]:
        target_gpu = cph.geometry.PointCloud(
            cph.utility.HostVector3fVector(pcd)
        )
        source_gpu = register(source_gpu, target_gpu)


    return source_gpu


pcd = do_point_cloud_registration(pcds)

ICP (GPU) [sec]: 0.026146888732910156
ICP (GPU) [sec]: 0.03260684013366699


## RBS

In [62]:
def get_bb_from_mesh(
    mesh:cph.geometry.TriangleMesh
) -> cph.geometry.AxisAlignedBoundingBox:

    AABB = mesh.get_axis_aligned_bounding_box()
    if AABB.is_empty():
        print('empty mesh detected')
        return None       
    else:
        return AABB
    

def perform_rbs_on_pointcloud_using_bb(obs, pcd):

    n = len(pcd.points)
    masks = list()
    meshes_list = list()
    for agent in AGENTS:
        print('\nAgent: %s' % agent)
        agent_obs = obs[agent]
        
        # get all meshes of the agent using fk
        meshes = get_mesh_using_forward_kinematics(
            agent_obs['joint_pos'],
            agent_obs['global_pos'],
            agent_obs['global_ang'],
        )
        meshes_list += meshes

        # find points within BBs and create masks
        start = time.time()
        for mesh in meshes:
            bb = get_bb_from_mesh(mesh)

            if bb is not None:
                mask = np.zeros(n, dtype=bool)

                indices_points_within_bb = np.asarray(
                    bb.get_point_indices_within_bounding_box(
                        pcd.points
                    ).cpu()
                )
                if len(indices_points_within_bb)>0:
                    mask[indices_points_within_bb] = True
                    masks.append(mask)

        elapsed_time = time.time() - start
        print("RBS (CPU+GPU) [sec]:", elapsed_time)


    mask = np.column_stack(tuple(masks)).any(axis=1)
    mask = ~mask # inverting mask to get pcd which were not within any BBs
    
    pcd_arr = np.asarray(pcd.points.cpu())
    NEIGHBOURS = 10
    pcd = cph.geometry.PointCloud(cph.utility.HostVector3fVector(pcd_arr[mask]))
    pcd = pcd.remove_statistical_outlier(nb_neighbors=NEIGHBOURS, std_ratio=10.0)
    return  pcd[0], meshes_list

rbs_pcd, meshes = perform_rbs_on_pointcloud_using_bb(obs, pcd)


Agent: panda0
empty mesh detected
empty mesh detected
RBS (CPU+GPU) [sec]: 0.02180027961730957

Agent: panda1
empty mesh detected
empty mesh detected
RBS (CPU+GPU) [sec]: 0.011800050735473633


In [63]:
if SHOW_CPH_VIZ:
    cph.visualization.draw_geometries([rbs_pcd]+meshes)

## Voxelization

In [64]:
def convert_pointcloud_to_voxels(pcd):
    cubic_size = 2.0
    # voxel_resolution = 100.0
    voxel_resolution = 30.0


    voxel_size = cubic_size / voxel_resolution

    # create voxel grid
    start = time.time()
    voxels = cph.geometry.VoxelGrid.create_from_point_cloud_within_bounds(
        pcd,
        voxel_size=cubic_size / voxel_resolution,
        min_bound=(-cubic_size / 2, -cubic_size / 2, -cubic_size / 2),
        max_bound=(cubic_size / 2, cubic_size / 2, cubic_size / 2),
    )
    elapsed_time = time.time() - start
    print("Voxelization (GPU) [sec]:", elapsed_time) # adding outlier removal adds ~25ms


    return voxels, voxel_size

voxels, voxel_size = convert_pointcloud_to_voxels(rbs_pcd)

Voxelization (GPU) [sec]: 0.0063419342041015625


In [65]:
if SHOW_CPH_VIZ:
    cph.visualization.draw_geometries([voxels])

In [67]:
radius = 0.03
VIZ = False

if VIZ:
    def voxel2sphere(voxels):
        global voxel_size, radius
        v = voxels.voxels.cpu()
        offset = voxels.get_min_bound()

        positions = np.array(list(v.keys()))

        positions = (positions - np.array([min(positions[:,0]), min(positions[:,1]), min(positions[:,2])]))
        positions = positions*voxel_size
        positions += (offset + voxel_size/2)

        spheres = cph.geometry.TriangleMesh.create_sphere(radius=0.01, resolution=5).translate(positions[0])
        for pos in positions[1:]:
            spheres += cph.geometry.TriangleMesh.create_sphere(
                radius=radius, 
                resolution=5).translate(pos)
        return spheres, positions

    spheres, positions  = voxel2sphere(voxels)    

    def meshColorscale(mesh):
        vertices = np.asarray(mesh.vertices.cpu())
        z_values = vertices[:, 2]

        # Normalize the z values to range [0, 1]
        z_min = z_values.min()
        z_max = z_values.max()
        z_norm = (z_values - z_min) / (z_max - z_min)

        # Create a color map (using a matplotlib colormap, e.g., 'viridis')
        cmap = plt.get_cmap('turbo')
        colors = cmap(z_norm)[:, :3]  # Get RGB values

        # Apply the colors to the mesh
        mesh.vertex_colors = cph.utility.Vector3fVector(colors)
        return mesh

    spheres = meshColorscale(spheres)

else:
    def voxel2sphere(voxels):
        global voxel_size
        start = time.time()
        v = voxels.voxels.cpu()
        offset = voxels.get_min_bound()

        positions = np.array(list(v.keys()))

        positions = (positions - np.array([min(positions[:,0]), min(positions[:,1]), min(positions[:,2])]))
        positions = positions*voxel_size
        positions += (offset + voxel_size/2)

        elapsed_time = time.time() - start
        print("Voxel2Sphere (CPU) [sec]:", elapsed_time) # adding outlier removal adds ~25ms

        return positions

    positions  = voxel2sphere(voxels)    

p = np.array(pcd.points.cpu())
print(min(p[:,0]),min(p[:,1]),min(p[:,2]))
print(max(p[:,0]),max(p[:,1]),max(p[:,2]))

print(min(positions[:,0]),min(positions[:,1]),min(positions[:,2]))
print(max(positions[:,0]),max(positions[:,1]),max(positions[:,2]))

Voxel2Sphere (CPU) [sec]: 0.02094292640686035
-1.4999754 -1.4988322 -1.555519e-05
1.4987465 1.4999558 1.4999748
-1.5 -1.5 -0.03333326801657677
1.5 1.5 1.5000000653167564


In [68]:
if VIZ:
    cph.visualization.draw_geometries([spheres])

## Approximation Output

In [69]:
import pickle
OUTPUT_PATH = '../outputs/dualarms_3cam/obstacle_approx.pkl'

with open(OUTPUT_PATH, 'wb') as f:
    pickle.dump(positions, f)

In [70]:
import yaml

class YamlStringObject():
    def __init__(self, obj:np.array):
        global radius
        self.pos = obj.tolist()
        self.radius = radius * 2.0
        self.vel = [0.0, 0.0, 0.0]
        self.isep = '  '
        self.osep = '      '

    def dump(self):
        string = f"{self.osep}- pos: {self.pos}"
        string += f"\n{self.osep}{self.isep}radius: {self.radius}"
        string += f"\n{self.osep}{self.isep}vel: {self.vel}"
        return string

yaml_str = "    obstacles:\n"

output_positions = positions + np.array([0.5, -0.2, 0.0])

for pos in output_positions:
    yaml_str += YamlStringObject(pos).dump()
    yaml_str += '\n'

# print(yaml_str)

In [71]:
OUTPUT_PATH = '../outputs/dualarms_3cam/obstacles.yaml'

with open(OUTPUT_PATH, 'w+') as f:
    f.write(yaml_str)

In [72]:
TASKS_FILE = '../outputs/dualarms_3cam/dual_arms_static3.yaml'
read_until = 1683

with open(TASKS_FILE, 'r') as file:
    contents = file.read()


OUTPUT_PATH = '../outputs/dualarms_3cam/dual_arms_static3.yaml'
with open(OUTPUT_PATH, 'w+') as f:
    f.write(contents[:1683]+"\n"+yaml_str)